# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

1. This paper focuses on an aspect of data cleaning known as data tidying, which is a way of organizing data in a structured, tabular format. This process makes it easier to deal with messy data without the need for complex tools, as well as allows for more efficient data manipulation, visualization, and analysis.
2. The tidy data standard makes initial data cleaning easier by preventing the "need to start from scratch." By providing a standard, consistent structure for organizing datasets, as well as a set of data analysis tools that work well together, it simplifies the initial exploration and analysis processes, allowing data analysts to focus on meaningful analysis rather than the logistics of the data.
3. The first sentence, "Like families, tidy datasets are all alike but every messy dataset is messy in its own way," is trying to reiterate the author's message that all tidy datasets follow a consistent structure, where each variable is a column and each observation is a row. In contrast, messy datasets can vary widely in how they are messy or disorganized; every messy dataset comes with its own unique structure and challenges. The second sentence, "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general," means that when looking at a particular dataset, we can identify the observations by looking at the values in rows and the variables by looking at the columns. However, it is more difficult to broadly define observations or variables because different data require different types (e.g. quantitative or qualitative variables) and structures.
4. Values: A dataset is a collection of values, or individual pieces of data, which are usually either numbers or strings. Variables: A variable contains all values that measure the same underlying attribute across units. Observations: An observation contains all values measured on the same unit across attributes.
5. Tidy data is a standard way of mapping the meaning of a dataset to its structure. Each variable forms a column, each observation forms a row, and each type of observational unit forms a table.
6. The five most common problems with messy datasets: 1) column headers are values, not variable names, 2) multiple variables are stored in one column, 3) variables are stored in both rows and columns, 4) multiple types of observational units are stored in the same table, 5) a single observational unit is stored in multiple tables. Table 4 is messy because the column headers are values that really represent the variable 'income'. This violates the tidy principle that each variable should have its own column. Melting a dataset is a way of tidying a dataset by turning columns into rows.
7. Table 11 is messy because it has multiple columns, i.e. d1-d8, that represent a single variable 'date'. Table 12a is molten because the date is now a single column. Table 12b is tidy because the element is split into separate variables with their respective values, and therefore, each variable is its own column and each row is a single observation.
8. The chicken-and-egg problem posits that tidy data is only as useful as the tools designed to manipulate it, meanwhile, tidy tools are built specifically for tidy data. This makes it easy to get stuck in a "local maxima" where changing data structures or tools will not improve workflow. The author hopes to see the field of research in data tidying grow to design more tools for data wrangling that optimize performance and usage. He suggests looking at human-computer interaction, cognitive sciences, multidimensional arrays, etc.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

Q2.1

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

First, I inspected the data frame using the .head() method and the .shape attribute to better visualize the data. Next, I printed the data type of the 'Price' variable and saw that it was of type 'object', which we do not want. Then, I printed all the unique values for 'Price' and saw that many were comma separated, which causes those values to be read in as strings. 

In [ ]:
df = pd.read_csv('./data/airbnb_hw.csv', low_memory=False)
df.head()
var = 'Price'
print('Shape: ', df.shape, '\n') # 30478 rows, 13 columns
print('Price Type (Before Cleaning): ', df[var].dtypes, '\n') # Type 'object'
print('Unique Values (Before Cleaning): ', df[var].unique(), '\n')

To clean the data, I removed commas from all the values using the .replace() method to prepare them to be converted into numeric values. Then, I used panda's .to_numeric() method to type cast all the values to integers and confirmed that this change was applied using the .dtype attribute and the .unique() method. Finally, I added all the null 'Price' values and found that there were 0 missing values.

In [ ]:
df[var] = df[var].str.replace(',', '') # Remove commas from all values
df[var] = pd.to_numeric(df[var], errors="coerce") # Type casting all values to int
print('Price Type (After Cleaning): ', df[var].dtype, '\n') # Confirm data type has changed to int
print('Unique Values (After Cleaning): ', df[var].unique(), '\n')

print('Total Missing: ', sum(df[var].isnull()), '\n') # 0 missing values

Q2.2

First, I inspected the data frame using the .unique() method and found that there was a nan category, and there were similar categories that could be consolidated under a single category (e.g. invalid, questionable, unconfirmed, unverified, and under investigation).

In [ ]:
sdf = pd.read_csv('./data/sharks.csv', low_memory=False)
sdf.head()
print('Unique Values: ', sdf['Type'].unique(), '\n') # nan value(s)
print(sdf['Type'].value_counts(), '\n') # Boat vs. boating vs. boatomg, invalid vs. unconfirmed vs. unverified vs. questionable vs. under investigation

I created a dummy variable, case_type, to hold the cleaned categories. I used the .replace() method to consolidate the 'uncertain' data (i.e. invalid, unconfirmed, unverified, questionable, and under investigation) under the nan category (i.e. treat them as missing), as well as consolidate the sea-related events under the category 'Sea Disaster'.

In [ ]:
case_type = sdf['Type']
case_type = case_type.replace(['Invalid', 'Questionable', 'Unconfirmed', 'Unverified', 'Under investigation'], np.nan)
case_type = case_type.replace(['Watercraft', 'Boat', 'Boating', 'Boatomg'], 'Sea Disaster')

sdf['Type'] = case_type
del case_type

print('Unique Values: ', sdf['Type'].unique(), '\n')
print(sdf['Type'].value_counts(), '\n')

Q2.3

In [188]:
pdf = pd.read_parquet('./data/justice_data.parquet', 'pyarrow') # pip installed pyarrow
pdf.head()
var = 'WhetherDefendantWasReleasedPretrial'
print(pdf[var].dtype, '\n') # integer
print(pdf[var].value_counts(), '\n') # 9 not defined in codebook
print('Total Missing (Before Cleaning): ', sum(pdf[var].isnull()), '\n')

released = pdf[var]
released = released.replace(9, np.nan)
pdf[var] = released
del released
print(pdf[var].value_counts(), '\n')
print('Total Missing (After Cleaning): ', sum(pdf[var].isnull()))

int64 

WhetherDefendantWasReleasedPretrial
1    19154
0     3801
9       31
Name: count, dtype: int64 

Total Missing (Before Cleaning):  0 

WhetherDefendantWasReleasedPretrial
1.0    19154
0.0     3801
Name: count, dtype: int64 

Total Missing (After Cleaning):  31


Q2.4

First, I inspected the 'SentenceTypeAllChargesAtConvictionInContactEvent' variable using the .value_counts() method and saw that there were 5 unique sentence types. With the help of the codebook, I saw that code 0 represents probation, 1 represents jail, 2 represents prison, 4 represents other, and 9 represents non-applicable. I also inspected the 'ImposedSentenceAllChargeInContactEvent' variable using the .unique() method and saw strings of numbers, as well as the empty string.

To clean the 'sentence' variable, I type casted the values to numeric to match the 'type' variable. Then, I summed the null values and found that there were 9053 missing values, 8779 of which were under code 4 and 274 of which were under code 9. So, I used the .mask() method to treat the values under code 4 as values under code 0, since code 4 includes cases where charges are dismissed, and to treat the values under code 9 as nan values for simplicity.

After cleaning, there were only 274 total missing values. 

In [190]:
sentence = pdf['ImposedSentenceAllChargeInContactEvent']
type = pdf['SentenceTypeAllChargesAtConvictionInContactEvent']

print(type.value_counts(), '\n') # 4 (other), 0 (probation), 1 (jail), 2 (prison), 9 (N/A)
print(sentence.unique(), '\n') # Strings, empty strings

sentence = pd.to_numeric(sentence, errors="coerce")
sentence_nan = sentence.isnull()
print('Total Missing: ', np.sum(sentence_nan), '\n') # 9053 missing values
print(pd.crosstab(sentence_nan, type), '\n') # 8779 in category 4, 274 in category 9

sentence = sentence.mask(type==4, 0)
sentence = sentence.mask(type==9, np.nan)

sentence_nan = sentence.isnull()
print('Total Missing: ', np.sum(sentence_nan), '\n') # 274 missing values
print(pd.crosstab(sentence_nan, type))

pdf['ImposedSentenceAllChargeInContactEvent'] = sentence
del sentence
del type

SentenceTypeAllChargesAtConvictionInContactEvent
4    8779
0    8720
1    4299
2     914
9     274
Name: count, dtype: int64 

[' ' '60' '12' '.985626283367556' '36' '6' '24' '5.91375770020534' '120'
 '72' '11.9917864476386' '0' '2.95687885010267' '84' '108' '300' '240'
 '180' '4' '96' '2' '54' '.328542094455852' '44' '5' '115' '132' '48'
 '258' '34' '76' '.164271047227926' '.131416837782341' '111' '9' '3'
 '1.97125256673511' '36.9856262833676' '.0657084188911704'
 '35.4928131416838' '106.492813141684' '8' '35' '18.3141683778234' '480'
 '32' '93' '234' '732' '1.16427104722793' '4.6570841889117' '21' '7'
 '4.49281314168378' '18' '600' '43.1642710472279' '179' '52' '30' '20'
 '192' '702' '14' '55' '53' '11.9055441478439' '114' '35.0061601642711'
 '68' '.657084188911704' '46.6242299794661' '102' '65' '200' '57'
 '24.3285420944559' '12.1642710472279' '117' '81.4928131416838'
 '22.4928131416838' '1980' '3.6570841889117' '56' '10' '2.79260780287474'
 '1' '47' '22' '1500' '40' '284' '11' '118

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?